In [1]:
import sqlite3
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState
from langgraph.checkpoint.sqlite import SqliteSaver

llm = init_chat_model("openai:gpt-5-nano")

conn = sqlite3.connect(
    "memory.db",
    check_same_thread=False,
)

config = {
    "configurable": {
        "thread_id": "11",
    },
}

In [2]:
class State(MessagesState):
    custom_stuff: str

graph_builder = StateGraph(State)

In [3]:
def chatbot(state: State):
    response = llm.invoke(state["messages"])
    return {
        "messages": [response],
    }

In [4]:
graph_builder.add_node("chatbot", chatbot)


graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile(
    checkpointer=SqliteSaver(conn),
)

In [ ]:
graph

In [5]:
result = graph.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "I live in Europe now. And the city I live in is Valencia.",
            },
        ]
    },
    config=config,
)

In [6]:
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

I live in Europe now. And the city I live in is Valencia.
================================== Ai Message ==================================

Nice! Valencia is a great city to live in. How can I help you settle in? Here are some quick pointers and options you might find useful.

Neighborhoods to know
- Ruzafa: lively, lots of bars and restaurants, very social.
- El Carmen (old town): historic streets, cafés, easy to walk to many sights.
- Benimaclet: student vibe, charming, several parks.
- Ciutat de les Arts i les Ciències area: modern, near the river and museums.
- Malvarrosa/Cabanyal: if you want beach life nearby.

Getting around
- Public transport: metro + light rail (Metrovalència) and buses (EMT Valencia). Look up a rechargeable transport card or passes.
- Biking: Valencia has decent bike lanes and you’ll see many cyclists.
- Walking: the historic center is very walkable.

Housing and costs
- Rent va

In [15]:
state_history = graph.get_state_history(config)

In [16]:
for state_snapshot in list(state_history):
    print(state_snapshot.next)
    print(state_snapshot.values["messages"])
    print("=" * 40 + "\n")

()
[HumanMessage(content='I live in Europe now. And the city I live in is Valencia.', additional_kwargs={}, response_metadata={}, id='bcbdfe8d-0325-4f0a-82ca-07c27698d27f'), AIMessage(content='Nice! Valencia is a great city to live in. How can I help you settle in? Here are some quick pointers and options you might find useful.\n\nNeighborhoods to know\n- Ruzafa: lively, lots of bars and restaurants, very social.\n- El Carmen (old town): historic streets, cafés, easy to walk to many sights.\n- Benimaclet: student vibe, charming, several parks.\n- Ciutat de les Arts i les Ciències area: modern, near the river and museums.\n- Malvarrosa/Cabanyal: if you want beach life nearby.\n\nGetting around\n- Public transport: metro + light rail (Metrovalència) and buses (EMT Valencia). Look up a rechargeable transport card or passes.\n- Biking: Valencia has decent bike lanes and you’ll see many cyclists.\n- Walking: the historic center is very walkable.\n\nHousing and costs\n- Rent varies by area, 

In [22]:
state_history = graph.get_state_history(config)

to_fork = list(state_history)[-2]

# to_fork.values["messages"]
to_fork.config

{'configurable': {'thread_id': '11',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0b88e8-b8be-66cd-8000-725c36031126'}}

In [23]:
from langchain_core.messages import HumanMessage

graph.update_state(
    to_fork.config,
    {
        "messages": [
            HumanMessage(
                content="I live in Europe now. And the city I live in is Seoul.",
                id="bcbdfe8d-0325-4f0a-82ca-07c27698d27f",
            )
        ]
    },
)

{'configurable': {'thread_id': '11',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0b8909-558e-6fc3-8001-08a41cd36b71'}}

In [24]:
forked_state = graph.get_state_history(
    {
        "configurable": {
            "thread_id": "11",
            "checkpoint_ns": "",
            "checkpoint_id": "1f0b8909-558e-6fc3-8001-08a41cd36b71",
        },
    }
)

list(forked_state)

[StateSnapshot(values={'messages': [HumanMessage(content='I live in Europe now. And the city I live in is Seoul.', additional_kwargs={}, response_metadata={}, id='bcbdfe8d-0325-4f0a-82ca-07c27698d27f')]}, next=('chatbot',), config={'configurable': {'thread_id': '11', 'checkpoint_ns': '', 'checkpoint_id': '1f0b8909-558e-6fc3-8001-08a41cd36b71'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-11-03T08:39:16.892024+00:00', parent_config={'configurable': {'thread_id': '11', 'checkpoint_ns': '', 'checkpoint_id': '1f0b88e8-b8be-66cd-8000-725c36031126'}}, tasks=(PregelTask(id='d143a8df-6cc8-abd1-2992-a66dde98db62', name='chatbot', path=('__pregel_pull', 'chatbot'), error=None, interrupts=(), state=None, result=None),), interrupts=())]

In [26]:
result = graph.invoke(
    None,
    {
        "configurable": {
            "thread_id": "11",
            "checkpoint_ns": "",
            "checkpoint_id": "1f0b8909-558e-6fc3-8001-08a41cd36b71",
        },
    },
)

for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

I live in Europe now. And the city I live in is Seoul.
================================== Ai Message ==================================

Just to check: Seoul is in South Korea, which is in Asia, not Europe. Do you mean one of the following?

- You live in a European city but you’re currently in Seoul (travelling or working there)?
- You meant you live in Europe and Seoul is relevant to you in another way (work, study, project)?
- Or you simply made a mistake?

If you clarify, I can tailor help. In the meantime, here are some options I can help with:

- Planning a trip to Seoul from Europe: visas, flights, neighborhoods to visit, transport tips.
- Moving to Seoul as a European: visa options, cost of living, housing, schools, expat communities.
- Learning Korean or basic phrases for travel and daily life.
- General info about Seoul’s culture and etiquette.
- Debugging data or wording if this was about a loc